<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [1]:
# notice: installing seaborn might takes a few minutes
#!pip install seaborn
#!pip install pydotplus 
#!pip install python-graphviz
#!conda install python-graphviz

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [ ]:
!wget -O ./loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

### Load Data From CSV File  

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

print('Train set: ', x_train.shape, y_train.shape)
print('Test set: ', x_test.shape, y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import metrics
max_k = x_train.shape[1] + 1
acc_list = np.zeros(max_k - 1)
std_list = np.zeros(max_k - 1)
knn_models = []
counfusionMt = []
for k in range(1, max_k):
    model = KNN(n_neighbors = k).fit(x_train, y_train)
    knn_models.append(model)
    y_hat = model.predict(x_test)
    acc_list[k - 1] = metrics.accuracy_score(y_test, y_hat)
    
    std_list[k - 1] = np.std(y_hat==y_test)/np.sqrt(y_hat.shape[0])


In [ ]:
plt.plot(range(1, max_k), acc_list, 'g')
plt.fill_between(range(1, max_k), acc_list - 1.65*std_list, acc_list + 1.65*std_list, alpha=0.15)
plt.legend(('Accuracy ', '95% estimation'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()
max_accr = acc_list.max()
max_index = acc_list.argmax() + 1

print("Maximum accurcy is %.3f%%" % (max_accr*100), "with k =", max_index)
KNN_model=knn_models[max_index]

# Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
print('Train set: ', x_train.shape, y_train.shape)
print('Test set: ', x_test.shape, y_test.shape)

In [ ]:
max_depths = int(x_train.shape[1] * 1.5)
train_accrs = np.zeros(max_depths - 1)
test_accrs = np.zeros(max_depths - 1)
mean_accrs = np.zeros(max_depths - 1)
for depth in range(1, max_depths):
    model_Decision_Tree = DTC(criterion='entropy', max_depth=depth)
    loanTree = model_Decision_Tree.fit(x_train, y_train)
    
    train_accrs[depth - 1] = metrics.accuracy_score(y_train, loanTree.predict(x_train))
    test_accrs[depth - 1] = metrics.accuracy_score(y_test, loanTree.predict(x_test))
    mean_accrs[depth - 1] = (train_accrs[depth - 1] + test_accrs[depth - 1]) / 2

plt.plot(range(1, max_depths), mean_accrs)
plt.plot(range(1, max_depths), train_accrs)
plt.plot(range(1, max_depths), test_accrs)
plt.legend(['Mean Accuracy', 'Train Accuracy', 'Test Accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Depth')
plt.tight_layout()
plt.show()

In [ ]:
depth_candidate = [3, 7, 10]
DTC_models = []
for depth in depth_candidate:
    DTC_models.append(DTC(criterion='entropy', max_depth=depth).fit(x_train, y_train))
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
accuracy = []
for model in DTC_models:
    accuracy.append(metrics.accuracy_score(y_test, model.predict(x_test)))
plt.scatter(depth_candidate, accuracy)
plt.ylabel('Accuracy')
plt.xlabel('Depth')
plt.tight_layout()
plt.show()
max_index = accuracy.index(max(accuracy))
print('max accuracy is %.3f%% with depth = ' %(accuracy[max_index] * 100), depth_candidate[max_index])

from sklearn.externals.six import StringIO
from sklearn import tree
import matplotlib.image as mpimg
import pydotplus
dot_data = StringIO()
filename = "DTC_output.png"
featureNames = Feature.columns[:]
targetNames = df['loan_status'].unique().tolist()
out=tree.export_graphviz(DTC_models[max_index], feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_train), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)

Decision_Tree_model = DTC_models[max_index]

# Support Vector Machine

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn import svm
X_svm = np.asarray(Feature)
y_svm = df['loan_status']
svm_x_train, svm_x_test, svm_y_train, svm_y_test = train_test_split(X_svm, y_svm, test_size = 0.3, random_state = 4)
model_svm = svm.SVC(kernel='rbf', gamma='auto')
model_svm.fit(svm_x_train, svm_y_train)
svm_y_hat = model_svm.predict(svm_x_test)

In [ ]:
cnf_matrix = confusion_matrix(svm_y_test, svm_y_hat, labels=['PAIDOFF', 'COLLECTION'])
np.set_printoptions(precision=2)

print (classification_report(svm_y_test, svm_y_hat))

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['PAIDOFF', 'COLLECTION'],normalize= False,  title='Confusion matrix')
SVM_model = model_svm

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
x_LR = X
y_LR = df['loan_status']
LR_x_train, LR_x_test, LR_y_train, LR_y_test = train_test_split(x_LR, y_LR, test_size=0.3, random_state=4)
print('Train set: ', LR_x_train.shape, LR_y_train.shape)
print('Test set: ', LR_x_test.shape, LR_y_test.shape)

In [ ]:
model_LR = LogisticRegression(C=10, solver='liblinear')
model_LR.fit(LR_x_train, LR_y_train)
LR_y_hat = model_LR.predict(LR_x_test)
LR_y_hat_prob = model_LR.predict_proba(LR_x_test)

In [ ]:
from sklearn.metrics import jaccard_similarity_score
LR_cnf_matrix = confusion_matrix(LR_y_test, LR_y_hat, labels=['PAIDOFF', 'COLLECTION'])
np.set_printoptions(precision=2)

print (classification_report(LR_y_test, LR_y_hat))
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(LR_cnf_matrix, classes=['PAIDOFF', 'COLLECTION'],normalize= False,  title='Confusion matrix')

LR_model = model_LR

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
#!wget -O ./loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_feature = test_df[['Principal','terms','age','Gender','weekend']]
test_feature = pd.concat([test_feature,pd.get_dummies(test_df['education'])], axis=1)
test_feature.drop(['Master or Above'], axis = 1,inplace=True)
test_x = np.asarray(test_feature)
test_y = test_df['loan_status']
labels = ['PAIDOFF', 'COLLECTION']

## Test and Evaluate the k-NN Model

In [ ]:
test_KNN = KNN_model.predict(test_x)
print (classification_report(test_y, test_KNN))
KNN_cnf_matrix = confusion_matrix(test_y, test_KNN, labels=labels)
plot_confusion_matrix(cnf_matrix, classes=labels, normalize= False,  title='Confusion matrix')
print('Jaccard : %.3f' % jaccard_similarity_score(test_y, test_KNN))

## Test and Evaluate the Decision Tree Model

In [ ]:
test_Decision_Tree = Decision_Tree_model.predict(test_x)
print (classification_report(test_y, test_Decision_Tree))
Decision_Tree_cnf_matrix = confusion_matrix(test_y, test_Decision_Tree, labels=labels)
plot_confusion_matrix(Decision_Tree_cnf_matrix, classes=labels, normalize= False,  title='Confusion matrix')
print('Jaccard : %.3f' % jaccard_similarity_score(test_y, test_Decision_Tree))

## Test and Evaluate the Logistic Regression Model

In [ ]:
test_SVM = SVM_model.predict(test_x)
print (classification_report(test_y, test_SVM))
SVM_cnf_matrix = confusion_matrix(test_y, test_SVM, labels=labels)
plot_confusion_matrix(SVM_cnf_matrix, classes=labels, normalize= False,  title='Confusion matrix')
print('Jaccard : %.3f' % jaccard_similarity_score(test_y, test_SVM))

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | 0.370       | 0.37        | NA      |
| Decision Tree      | 0.593       | 0.59        | NA      |
| SVM                | 0.759       | 0.76        | NA      |
| LogisticRegression | 0.741       | 0.74        | 8.954       |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>